#### Connect the MySQL database ####

In [35]:
import mysql.connector
import pandas as pd
import numpy as np
import torch

In [36]:
connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='password',
            database='PPT'
            )

In [12]:
query = """ SELECT * FROM posts_details"""

In [13]:
df = pd.read_sql(query, connection)

/var/folders/xp/1gy0dsh531s017q00bdzwmgw0000gn/T/ipykernel_80978/1848774206.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [14]:
df.head()

,id,title,author,date,content,comments
0,1,Re: [問卦] 以前當兵都是一起洗澡的嗎？？？,ai2311 (),Fri Apr 19 21:18:58 2024,"['※ 引述《gn505250 (dwas356916)》之銘言：', ': 聽說以前當兵都...",
1,2,[問卦] 你們股票都有賺到吧？,opemhood (大漢堡),Fri Apr 19 21:21:24 2024,欸欸最近八年來，股市一直破新高這八年，鄉民都有賺到錢吧？一定有賺到錢吧？有沒有這方面的八卦？--,
2,3,[問卦] 歸０是什麼感覺,vigle5566 (威哥),Fri Apr 19 21:20:11 2024,搭給賀 我天橋下肥宅拉阿肥友人是一位玩遊戲會怒吼亂搓的豬角色的操控很弱 技能使出來的叭叭會被...,
3,4,[問卦] 凱格爾訓練機幹嘛用的？,cowardlyman (有功夫無懦夫),Fri Apr 19 21:17:50 2024,本肥被推薦購買這個東西這個東西是要訓練什麼看那顆球的位置是要訓練肛門讓括約肌變有力嗎？有卦嗎？--,
4,5,"[問卦] 台灣空軍飛官,有人遇過UFO嗎?",lelena (太陽神-RA),Fri Apr 19 21:17:26 2024,"遇到UFO的飛行員,都是國外的飛行員,台灣飛行員也是每天在空中飛,不知道有沒有台灣飛官遇過不...",


#### Install NLP model transalte chinese to English

In [1]:
# first version example
from transformers import pipeline

# 创建翻译 pipeline，指定模型
translator = pipeline("translation", model="facebook/m2m100_418M")
# 输入中文文本，进行翻译
chinese_text = "這些垃圾國家美國真的不用浪費時間"
translated_text = translator(chinese_text, src_lang="zh", tgt_lang="en")

# 输出翻译后的英文文本
print(translated_text)

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

[{'translation_text': 'These waste countries do not really waste time.'}]


In [25]:
# Download the model
#from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Define the model and tokenizer
#model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
#tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
#model.save_pretrained('./NLP_Translate_Model/m2m100_418M_model')
#tokenizer.save_pretrained('./NLP_Translate_Model/m2m100_418M_tokenizer')

In [26]:
from transformers import pipeline, M2M100ForConditionalGeneration, M2M100Tokenizer

# Load the model and tokenizer from local directory
model = M2M100ForConditionalGeneration.from_pretrained('./NLP_Translate_Model/m2m100_418M_model')
tokenizer = M2M100Tokenizer.from_pretrained('./NLP_Translate_Model/m2m100_418M_tokenizer')

# Create a translation pipeline using the local model and tokenizer
translator = pipeline("translation", model=model, tokenizer=tokenizer)
def translate(text):
    # Input Chinese text and translate
    chinese_text = text
    translated_text = translator(chinese_text, src_lang="zh", tgt_lang="en")

    # Output the translated English text
    return translated_text[0]['translation_text']

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
chinese_text = "欸欸最近八年來，股市一直破新高這八年，鄉民都有賺到錢吧？一定有賺到錢吧？有沒有這方面的八卦？"
translate_chinese_text = translate(chinese_text)
print(translate_chinese_text)

In the last eight years, the stock market has broken up these eight years, the country’s people have earned money, right?


#### Sentiment Analysis

In [ ]:
# Download the model
#from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Define the model and tokenizer
#model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
#tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Save the model and tokenizer locally
#model.save_pretrained('./NLP_Sentiment_Analysis/sentiment_analysis_model')
#tokenizer.save_pretrained('./NLP_Sentiment_Analysis/sentiment_analysis_tokenizer')


In [33]:
from transformers import pipeline, DistilBertForSequenceClassification, DistilBertTokenizer

# Load the model and tokenizer from local directory
model = DistilBertForSequenceClassification.from_pretrained('./NLP_Sentiment_Analysis/sentiment_analysis_model')
tokenizer = DistilBertTokenizer.from_pretrained('./NLP_Sentiment_Analysis/sentiment_analysis_tokenizer')

# Create a sentiment analysis pipeline using the local model and tokenizer
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Input text (assuming `translated_text` is the output from your translation pipeline)
def sentiment_analysis(text):
    print(text)
    return classifier(text)


In [34]:
sentiment_analysis(translated_text[0]['translation_text'])

In the last eight years, the stock market has broken up these eight years, the country’s people have earned money, right?


[{'label': 'NEGATIVE', 'score': 0.9662074446678162}]

### Add new Column for the sentiment analysis result

In [71]:
import re

In [2]:
def classify_content(text):
    text = re.sub(r"[^\u4e00-\u9fff\d.a-zA-Z%+\-。！？，、；：（）【】《》“”‘’]", '', text)  # 去除特殊字符
    return text

#### Content summarize